In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv('./datasets/features.csv')
test=pd.read_csv('./datasets/test.csv')
y=data['radiant_win']

remove=['start_time']

<hr>
2- Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

Print columns with missing data

In [3]:
N=len(data)
for col in data.columns:
    if data[col].count()<N:
        print col,data[col].count()

first_blood_time 77677
first_blood_team 77677
first_blood_player1 77677
first_blood_player2 53243
radiant_bottle_time 81539
radiant_courier_time 96538
radiant_flying_courier_time 69751
radiant_first_ward_time 95394
dire_bottle_time 81087
dire_courier_time 96554
dire_flying_courier_time 71132
dire_first_ward_time 95404


some values are omitted of the following features : 
    <b>first_blood_time , first_blood_team ,first_blood_player1 , first_blood_player2 </b>
becuase the event didn't happen in the first 5 minuites

<hr>
1- Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).

In [4]:
#Find extra columns in training set
extraTrainColumns=[col for col in data.columns if col not in test.columns]
print extraTrainColumns

['duration', 'radiant_win', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire']


In [5]:
#Remove extra columns except target column
extraTrainColumns.remove('radiant_win')
print extraTrainColumns
data.drop(extraTrainColumns,axis=1,inplace=True)

['duration', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire']


<hr>
3- Замените пропуски на нули с помощью функций fillna (). На самом деле этот способ является предпочтительным для логистической регрессии, так как он позволяет пропущенному значению не вносить никакого вклада в предсказание. Для тех, кто хочет, чтобы сменить место, где можно было бы отправиться в гостиную с пропусками в отдельную ветвь дерева. Также есть и другие подходы - например, замена пропуска на среднее значение признака. Мы не требуем этого в задании, но при желании попробуйте разные подходы к обработке пропусков и сравните их между собой.

In [6]:
data=data.fillna(0)
test=test.fillna(0)

<hr>
4- Какой столбец содержит целевую переменную? Запишите его название.

radiant_win
<hr>

In [7]:
data.to_csv('train.csv')
test.to_csv('test.csv')

<hr>
### Heros-use bag of words

In [8]:
heros=pd.read_csv("./datasets/dictionaries/heroes.csv")
NumHeros=heros['id'].max()

In [9]:
categorical=["r{}_hero".format(i) for i in range(1,6)]
categorical+=["d{}_hero".format(i) for i in range(1,6)]
categorical.append('lobby_type')
clean_data=data.drop(categorical,axis=1)
clean_test=test.drop(categorical,axis=1)

In [10]:
pick = np.zeros((data.shape[0], NumHeros))
for i,ind in enumerate(data.index):
    for h in range(1,6):
        pick[i][data["r{}_hero".format(h)][ind]-1]=1
        pick[i][data["d{}_hero".format(h)][ind]-1]=-1
pick=pd.DataFrame(pick)

picktest=np.zeros((test.shape[0], NumHeros))
for i,ind in enumerate(test.index):
    for h in range(1,6):
        picktest[i][test["r{}_hero".format(h)][ind]-1]=1
        picktest[i][test["d{}_hero".format(h)][ind]-1]=-1
picktest=pd.DataFrame(picktest)

In [11]:
data_herobag=pd.concat([clean_data,pick],axis=1)
test_herobag=pd.concat([clean_test,picktest],axis=1)

In [12]:
data_herobag.to_csv('train_herobag.csv')
test_herobag.to_csv('test_herobag.csv')

<hr>

In [13]:
cols=['level','xp','gold','lh','deaths','kills','items']

In [23]:
data2=data_herobag
X=pd.DataFrame([])
for col in cols:
    rv=["r{}_{}".format(ind,col) for ind in range(1,6)]
    dv=["d{}_{}".format(ind,col) for ind in range(1,6)]
    hrmean=data[rv].mean(axis=1)
    hrmin=data[rv].max(axis=1)
    hrmax=data[rv].min(axis=1)
    hrsum=data[rv].sum(axis=1)
    hdmean=data[dv].mean(axis=1)
    hdmin=data[dv].max(axis=1)
    hdmax=data[dv].min(axis=1)
    hdsum=data[dv].sum(axis=1)
    data2=data2.drop(rv+dv,axis=1)
    X=pd.concat([X,hrmean,hrmin,hrmax,hdmean,hdmin,hdmax],axis=1)
data2

,match_id,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,...,103,104,105,106,107,108,109,110,111,112
0,0,1430198770,5,2098,1489,20,0,0,7,3,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1430220345,4,1188,1033,9,0,1,12,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,1430227081,4,1319,1270,22,0,0,12,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,1430263531,4,1779,1056,14,0,0,5,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,1430282290,4,1431,1090,8,1,0,8,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,1430284186,5,1961,1461,19,0,1,6,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,8,1430293701,3,967,1136,7,1,0,8,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,9,1430299335,5,2117,1252,16,0,0,6,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,11,1430308974,5,1527,906,10,0,1,7,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,12,1430316105,5,1651,1060,14,0,1,10,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
test2=test_herobag
X2=pd.DataFrame([])
for col in cols:
    rv=["r{}_{}".format(ind,col) for ind in range(1,6)]
    dv=["d{}_{}".format(ind,col) for ind in range(1,6)]
    hrmean=test[rv].mean(axis=1)
    hrmin=test[rv].max(axis=1)
    hrmax=test[rv].min(axis=1)
    hrsum=test[rv].sum(axis=1)
    hdmean=test[dv].mean(axis=1)
    hdmin=test[dv].max(axis=1)
    hdmax=test[dv].min(axis=1)
    hdsum=test[dv].sum(axis=1)
    test2=test2.drop(rv+dv,axis=1)
    X2=pd.concat([X2,hrmean,hrmin,hrmax,hdmean,hdmin,hdmax],axis=1)
test2

,match_id,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,...,103,104,105,106,107,108,109,110,111,112
0,6,1430287923,4,1103,1089,8,0,1,9,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,1430293357,2,556,570,1,0,0,9,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10,1430301774,2,751,808,1,0,0,13,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,13,1430323933,3,708,903,1,1,1,11,2,...,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0
4,16,1430331112,4,1259,661,4,0,0,9,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,18,1430334264,5,2067,1549,15,0,0,8,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,19,1430334995,4,1862,1487,24,0,1,5,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7,24,1430349466,5,1991,1983,19,2,0,10,4,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,33,1430366484,2,517,500,0,0,1,9,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,37,1430375099,3,1060,691,5,0,0,6,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
data2=pd.concat([data2,X],axis=1)
test2=pd.concat([test2,X2],axis=1)

In [26]:
data2

,match_id,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,...,2,3,4,5,0,1,2,3,4,5
0,0,1430198770,5,2098,1489,20,0,0,7,3,...,0,0.2,1,0,7.4,11,4,6.6,9,4
1,1,1430220345,4,1188,1033,9,0,1,12,4,...,0,0.4,2,0,7.6,12,5,8.2,12,5
2,2,1430227081,4,1319,1270,22,0,0,12,3,...,0,0.2,1,0,7.8,12,6,8.4,10,7
3,3,1430263531,4,1779,1056,14,0,0,5,2,...,0,0.0,0,0,5.8,6,5,7.4,10,6
4,4,1430282290,4,1431,1090,8,1,0,8,2,...,0,0.2,1,0,8.4,11,7,8.0,9,7
5,5,1430284186,5,1961,1461,19,0,1,6,2,...,0,0.4,1,0,6.0,9,4,9.8,13,7
6,8,1430293701,3,967,1136,7,1,0,8,4,...,0,0.8,1,0,9.2,12,8,11.0,16,8
7,9,1430299335,5,2117,1252,16,0,0,6,3,...,0,0.0,0,0,6.2,8,4,7.4,9,7
8,11,1430308974,5,1527,906,10,0,1,7,3,...,0,0.4,1,0,6.0,8,4,7.6,10,5
9,12,1430316105,5,1651,1060,14,0,1,10,3,...,0,0.4,1,0,7.4,10,4,7.8,11,6


In [27]:
data2.to_csv('train_2.csv')
test2.to_csv('test_2.csv')